## Step 4 Create Network Graph, Assign Link Costs, and Run BikewaySim

1. Process network spatial data into a routable network graph format
2. Reconcile networks into one through node and link overlap conflation
3. __Create final network graph and calculate link costs__
4. Create OD tables
5. Run BikewaySim

In [1]:
from pathlib import Path
import geopandas as gpd

from network_filter import *
from prepare_network import *

## Export Filepath

In [2]:
#transitsim
studyarea_name = 'transitsim'
export_fp = Path.home() / 'Documents/TransitSimData/networks'

# studyarea_name = 'bikewaysim'
# export_fp = Path.home() / Path(f'Documents/NewBikewaySimData/{studyarea_name}')


## Link Costs Dictionary
Dict keys must correspond to column names in links GeoDataFrame.
The links cost funciton is off this format:
$$ linkcost = linkdistance * (1-\sum \beta x) $$

Negative attributes **decrease** impedance  
Positive attributes **increase** impedance

In [3]:
costs0 = {
    'bl':-0.05,
    'pbl':-0.95,
    'mu':-0.95,
    '<25mph':0,
    '25-30mph':0.30,
    '>30mph':1,
    '1lpd':0,
    '2-3lpd':0.50,
    '>4lpd':1     
    }

In [4]:
costs1 = {
    'bl':-0.25,
    'pbl':-0.60,
    'mu':-0.60,
    '<25mph':0,
    '25-30mph':0.50,
    '>30mph':1.5,
    '1lpd':0,
    '2-3lpd':0.50,
    '>4lpd':1.5     
    }

In [5]:
costs2 = {
    'bl':-0.05,
    'pbl':-0.5,
    'mu':-0.5,
    '<25mph':0,
    '25-30mph':0.5,
    '>30mph':4,
    '1lpd':0,
    '2-3lpd':1,
    '>4lpd':4     
    }

In [6]:
cost_dicts = {
    'costs0':costs0,
    'costs1':costs1,
    'costs2':costs2
}

In [7]:
#import links you want to use (these are the trb 2023 links)
#links = gpd.read_file(export_fp / Path('reconciled_network.gpkg'),layer='here_trb')
#nodes = gpd.read_file(export_fp / Path('reconciled_network.gpkg'),layer='here_trb_nodes')

links = gpd.read_file(export_fp / 'reconciled_network.gpkg',layer='links')
nodes = gpd.read_file(export_fp / 'reconciled_network.gpkg',layer='nodes')

#prepare network
links, nodes = prepare_network(links,nodes,spd_mph=8)


In [8]:
#if just time impedance, export as is
nodes.to_file(export_fp/'final_network.gpkg',layer='nodes',driver='GPKG')
links.to_file(export_fp/'final_network.gpkg',layer='links',driver='GPKG')

In [9]:
#repeat for low stress network
links = gpd.read_file(export_fp / 'lowstress_reconciled_network.gpkg',layer='links')
nodes = gpd.read_file(export_fp / 'lowstress_reconciled_network.gpkg',layer='nodes')

#prepare network
links, nodes = prepare_network(links,nodes,spd_mph=8)

#if just time impedance, export as is
nodes.to_file(export_fp/'final_network.gpkg',layer='lowstress_nodes',driver='GPKG')
links.to_file(export_fp/'final_network.gpkg',layer='lowstress_links',driver='GPKG')

In [22]:
for key in cost_dicts.keys():

    #other costs to test
    links = link_costs(links, cost_dicts[key], key)

    #make fp organizing
    if not (export_fp / Path(key)).exists():
        (export_fp / Path(key)).mkdir()

    positive_cols = [x for x in cost_dicts[key].keys() if cost_dicts[key][x] < 0]
    nochange_cols = [x for x in cost_dicts[key].keys() if cost_dicts[key][x] == 0]
    negative_cols = [x for x in cost_dicts[key].keys() if cost_dicts[key][x] > 0]

    positive = (links[positive_cols] == 1).any(axis=1)
    negative = (links[negative_cols] == 1).any(axis=1)
    no_change = (links[nochange_cols] == 1).any(axis=1)

    only_positive = positive & -negative
    only_negative = -positive & negative
    mixed = positive & negative #& -no_change
    only_no_change = no_change & -positive & -negative

    only_positive = links[only_positive]
    only_negative = links[only_negative]
    mixed = links[mixed]
    no_change = links[only_no_change]

    only_positive.to_file(export_fp / key / 'link_cost_changes.gpkg',layer='Only Decreased')
    only_negative.to_file(export_fp / key / 'link_cost_changes.gpkg',layer='Only Increased')
    mixed.to_file(export_fp / key / 'link_cost_changes.gpkg',layer='Mixed')
    no_change.to_file(export_fp / key / 'link_cost_changes.gpkg',layer='No Change')

#export
nodes.to_file(export_fp/'final_network.gpkg',layer='nodes',driver='GPKG')
links.to_file(export_fp/'final_network.gpkg',layer='links',driver='GPKG')

In [23]:
#links[-links['A_B'].isin(pd.concat([only_positive,only_negative,mixed,no_change])['A_B'])][list(costs0.keys())]#.drop_duplicates()

## Improvements

In [24]:
#extend protected bike lane on 10th st
improvements = links.copy()
changes = ['10th Street Northwest','10th Street Northeast']
improvements.loc[improvements['name'].isin(changes),'pbl'] = 1

#change that one link back
improvements.loc[improvements['A_B']=='9543446970_69614574','pbl'] = 0
improvements.loc[improvements['A_B']=='69614574_9543446970','pbl'] = 0


#export that one
tenth_st  = improvements[improvements['pbl'] != links['pbl']].copy()

tenth_st.geometry = tenth_st.buffer(300)
tenth_st = tenth_st.dissolve()
tenth_st.to_file(export_fp/'network_improvements.gpkg',layer='New Protected Bike Lane')

#just pbl
just_pbl = improvements.copy()


In [25]:
def add_ref_ids(links,nodes):
    '''
    This function adds reference columns to links from the nodes id column
    '''
    for_matching = links.copy()
    #make the first point the active geometry
    for_matching['pt_geometry'] = for_matching.apply(start_node_geo, geom='geometry', axis=1)
    for_matching.set_geometry('pt_geometry',inplace=True)
    for_matching.drop(columns='geometry',inplace=True)
    #find nearest node from starting node and add to column
    links['A'] = ckdnearest(for_matching,nodes,return_dist=False)['N']
    
    #repeat for end point
    for_matching = links.copy()
    #make the first point the active geometry
    for_matching['pt_geometry'] = for_matching.apply(end_node_geo, geom='geometry', axis=1)
    for_matching.set_geometry('pt_geometry',inplace=True)
    for_matching.drop(columns='geometry',inplace=True)
    #find nearest node from starting node and add to column
    links['B'] = ckdnearest(for_matching,nodes,return_dist=False)['N']

    #check for missing reference ids
    if links['A'].isnull().any() | links['B'].isnull().any():
        print("There are missing reference ids")
    else:
        print("Reference IDs successfully added to links.")
    return links

In [26]:
#bring in new feature and add to old links
new = gpd.read_file(export_fp/'network_improvements.gpkg',layer='morningside_path')
morningside_path = new.copy()
morningside_path.geometry = morningside_path.buffer(300)
morningside_path = morningside_path.dissolve()
morningside_path.to_file(export_fp / "network_improvements.gpkg", layer = 'New Multi-Use Path')

#add ref ids ONLY if no new nodes are added
new = add_ref_ids(new,nodes)

#make reverse
new = create_reverse_links(new)
new['dist'] = new.length
new['mins'] = new['dist'] / 5280 / 8 * 60

#add to network with 10st pbl
improvements = pd.concat([improvements,new],ignore_index=True).reset_index()

#just add path
just_path = links.copy()
just_path = pd.concat([just_path,new],ignore_index=False).reset_index()

#TODO incorporate other methods for adding new links quickly
#needs to be able to split existing links by line ends
#new ids for new nodes keep ids for split

c:\Users\tpassmore6\Anaconda3\envs\geo-env\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
c:\Users\tpassmore6\Anaconda3\envs\geo-env\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


Reference IDs successfully added to links.


In [27]:
#redo costs
for key in cost_dicts.keys():
    #other costs to test
    just_pbl = link_costs(just_pbl, cost_dicts[key], key)
    just_path = link_costs(just_path, cost_dicts[key], key)
    improvements = link_costs(improvements, cost_dicts[key], key)

#export
just_pbl.to_file(export_fp / 'final_network.gpkg',layer='just_pbl_links')
just_path.to_file(export_fp / 'final_network.gpkg',layer='just_path_links')
improvements.to_file(export_fp / 'final_network.gpkg',layer='improved_links')

In [28]:
# #%% NCST Project
# road_links = gpd.read_file(Path('processed_shapefiles/osm/osm_marta_network.gpkg'),layer='road_links')
# road_nodes = gpd.read_file(Path('processed_shapefiles/osm/osm_marta_network.gpkg'),layer='road_nodes')

# bike_links = gpd.read_file(Path('processed_shapefiles/osm/osm_marta_network.gpkg'),layer='bike_links')
# bike_nodes = gpd.read_file(Path('processed_shapefiles/osm/osm_marta_network.gpkg'),layer='bike_nodes')

# roadbike_links = road_links.append(bike_links)
# roadbike_nodes = road_nodes.append(bike_nodes).drop_duplicates()

# roadbike_links.rename(columns={'osm_A':'A','osm_B':'B'},inplace=True)

# links, nodes = prepare_network(roadbike_links,roadbike_nodes,link_costs=True)

# print('exporting...')
# links.to_file(Path.home() / Path('Documents/TransitSimData/Data/osm_network.gpkg'),layer='links',driver='GPKG')
# nodes.to_file(Path.home() / Path('Documents/TransitSimData/Data/osm_network.gpkg'),layer='nodes',driver='GPKG')
# print('done.')